In [3]:
import os
import json as json_loader


In [7]:
print(os.listdir("./020JErJMvVZ"))
with open("./020JErJMvVZ/ parsed.json","r") as f:
    json = json_loader.load(f)

['figure', 'reference_simplified.bib', ' parsed.json', 'arxiv.tex', 'arxiv.bbl']


In [18]:
print(json["blocks"][0]["c"])

[{'t': 'Span', 'c': [['', [], []], [{'t': 'Str', 'c': 'Ye'}, {'t': 'Space'}, {'t': 'Emph', 'c': [{'t': 'Str', 'c': 'et'}, {'t': 'Space'}, {'t': 'Str', 'c': 'al.'}]}, {'t': 'Str', 'c': ':'}, {'t': 'Space'}, {'t': 'Str', 'c': 'Learning'}, {'t': 'Space'}, {'t': 'Str', 'c': 'Continuous'}, {'t': 'Space'}, {'t': 'Str', 'c': 'Grasping'}, {'t': 'Space'}, {'t': 'Str', 'c': 'Function'}, {'t': 'Space'}, {'t': 'Str', 'c': 'with'}, {'t': 'Space'}, {'t': 'Str', 'c': 'a'}, {'t': 'Space'}, {'t': 'Str', 'c': 'Dexterous'}, {'t': 'Space'}, {'t': 'Str', 'c': 'Hand'}, {'t': 'Space'}, {'t': 'Str', 'c': 'from'}, {'t': 'Space'}, {'t': 'Str', 'c': 'Human'}, {'t': 'Space'}, {'t': 'Str', 'c': 'Demonstrations'}]]}]


In [26]:
def extract_math(json_data):
    """
    Extract all math elements (inline and display) from the JSON data.
    """
    math_expressions = []

    def extract_from_block(block):
        # Check block types for math
        if isinstance(block, dict):
            if block.get("t") == "Math":
                math_expressions.append(block.get("c", ["", ""])[1])
            elif "c" in block and isinstance(block["c"], list):
                for item in block["c"]:
                    extract_from_block(item)
        elif isinstance(block, list):
            for item in block:
                extract_from_block(item)

    # Traverse through the blocks
    if "blocks" in json_data:
        for block in json_data["blocks"]:
            extract_from_block(block)

    return math_expressions

In [136]:

def extract_math_with_chapters(json_path):
    """
    CHATGPT GENERATED 

    Extract math expressions and their associated chapters.
    
    """
    with open(json_path,"r") as f:
        json_data = json_loader.load(f)
        
    chapter = [0]
    math_with_chapters = []

    def extract_from_block(block):
        nonlocal chapter
        # Check block types
        if isinstance(block, dict):
            if block.get("t") == "Header":
                # If it's a chapter header, update the chapter title
                level, _ = block.get("c", [])[0], block.get("c", [])[1]

                if len(chapter) >= level:
                    chapter[level-1] += 1
                    chapter = chapter[:level]
                else:
                    chapter.append(0)

            elif block.get("t") == "Math":
                # Extract math expression and associate it with the current chapter
                math_expr = block.get("c", ["", ""])[1]
                chapter_str = ""
                for c in chapter:
                    chapter_str += str(c) + "."
                chapter_str = chapter_str[:-1]
                math_with_chapters.append((chapter_str, math_expr))
            elif "c" in block and isinstance(block["c"], list):
                # Recursively process nested blocks
                for item in block["c"]:
                    extract_from_block(item)
        elif isinstance(block, list):
            for item in block:
                extract_from_block(item)

    # Traverse through the blocks
    if "blocks" in json_data:
        for block in json_data["blocks"]:
            extract_from_block(block)

    return math_with_chapters

In [138]:
extract_math_with_chapters("020JErJMvVZ/ parsed.json")

[('0', '^1'),
 ('1', 'z'),
 ('1', 't'),
 ('1', 't'),
 ('1', 't'),
 ('1', 'z'),
 ('1', 'z'),
 ('3.1',
  '\\begin{aligned}\n\\min_{q_t}& \\quad \\sum^{N}_{i=0}||\\mathcal{J}_i(q_t)-j_i||^2+\\lambda ||\\mathcal{J}_i(q_t)-\\mathcal{J}_i(q_{t-1})||^2 \\\\\n\\text{s.t.}& \\quad q_{lower} \\leq q_t \\leq q_{upper},\n\\end{aligned}'),
 ('3.1', 'q_t'),
 ('3.1', 't'),
 ('3.1', '\\mathcal{J}_i'),
 ('3.1', 'j_i'),
 ('3.1', 'q_{lower}'),
 ('3.1', 'q_{upper}'),
 ('3.1', 'q_t'),
 ('3.1', 'q_{t-1}'),
 ('3.1', 'q_0=\\frac{1}{2}(q_{lower}+q_{upper})'),
 ('3.2', '\\mathcal{P}^{o} \\in \\mathbb{R}^{N \\times 3}'),
 ('3.2', 'N'),
 ('3.2', '\\{ q_t \\}, t \\in \\{0, 1, \\cdots, T\\}'),
 ('3.2', 'T'),
 ('3.2', '\\mathcal{F}^{o} \\in \\mathbb{R}^{1024}'),
 ('3.2', '\\{\\mathcal{F}^{h}_t\\} \\in \\mathbb{R}^{256}'),
 ('3.2', '\\mathcal{F}^{oh}'),
 ('3.2', '\\mu \\in \\mathbb{R}^{256}'),
 ('3.2', '\\sigma^{2} \\in \\mathbb{R}^{256}'),
 ('3.2', '\\mathcal{Q}\\left(z \\mid \\mu, \\sigma^{2}\\right)'),
 ('3.2', 'z

In [140]:
import xml.etree.ElementTree as ET

def parse_latexml(xml_file):
    """
    Parse the XML file generated by LaTeXML.
    """
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()
        return root
    except ET.ParseError as e:
        print(f"Error parsing XML: {e}")
        return None

def extract_math_with_chapters_from_xml(root):
    """
    Extract math expressions and their associated chapters from the LaTeXML XML structure.
    """
    math_with_chapters = []
    current_chapter = None

    # Iterate through all elements in the XML tree
    for elem in root.iter():


        # Detect chapters (assuming <chapter> or <section> elements mark chapters)
        if "section" in elem.tag.lower() or "chapter" in elem.tag.lower():
            # Update the current chapter (using title or similar elements)
            attr = elem.attrib
            for key,val in attr.items():
                if "id" in key:
                    current_chapter = val.replace("S","")

        # Detect math elements (inline or display math)
        elif "math" in elem.tag.lower():
            if elem.attrib != {}:
                math_with_chapters.append((current_chapter, elem.attrib["tex"]))

    return math_with_chapters

# Step 1: Parse the XML file
root = parse_latexml("09I1M8YRJBR/parsed.xml")

# Step 2: Extract math expressions with chapter information
math_with_chapters = extract_math_with_chapters_from_xml(root)

# Step 3: Print or save the extracted math with chapters
print("Extracted Math Expressions by Chapter:")
for e in  math_with_chapters:
    print(e)


Extracted Math Expressions by Chapter:
('1', '5')
('1', 't')
('1', 'x')
('1', 'u')
('1', 'h')
('1', '\\thicksim')
('1', 't')
('1', 'x')
('1', 't')
('1', 'x')
('3.1', 't')
('4', '10^{-3}')
('4', '10^{-3}')
('4', 'a\\rightarrow u')
('4', 'h\\rightarrow u')
('4', 'u\\rightarrow a')
('4', 'u\\rightarrow h')
('4', 'x_{p},T\\rightarrow\\theta,x_{a}')
('4', '\\theta,x_{a}\\rightarrow x_{p},T')
('4.1', '\\displaystyle\\nabla\\left(a(x)\\nabla u(x)\\right)=f(x)\\text{, }x\\in(0,1)^{2},')
('4.1', '\\displaystyle\\nabla\\left(a(x)\\nabla u(x)\\right)')
('4.1', '\\displaystyle=f(x)\\text{, }x\\in(0,1)^{2},')
('4.1', '\\displaystyle u(x)=0\\text{, }x\\in\\partial(0,1)^{2},')
('4.1', '\\displaystyle u(x)')
('4.1', '\\displaystyle=0\\text{, }x\\in\\partial(0,1)^{2},')
('4.1', 'a(x)')
('4.1', 'f(x)=1')
('4.1', 'h')
('4.1', 'u')
('4.1', '\\displaystyle\\frac{\\partial h}{\\partial t}=-\\frac{\\partial hu}{\\partial x}%\n\\text{, }\\quad\\frac{\\partial hu}{\\partial t}=-\\frac{\\partial(u^{2}h+0.5gh^{2